# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [3]:
X_test, y_test = test_df()

In [4]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2004,chrysler,pacifica,awd,NaN,automatic,2c8gf68494r646414,nj,2.5,142075.0,silver,black,lester glenn buick,2525,2800,Wed Dec 31 2014 09:30:00 GMT-0800 (PST)
1,2009,Acura,TL,SH-AWD,Sedan,automatic,19uua96549a006477,oh,4.4,117084.0,black,brown,pauley motor car co preowned vehicles llc,12550,13000,Tue Jan 13 2015 09:30:00 GMT-0800 (PST)
2,2012,Honda,Accord,LX,Sedan,automatic,1hgcp2f33ca014965,ca,3.9,34316.0,silver,gray,barber honda,12050,13700,Tue Feb 10 2015 04:30:00 GMT-0800 (PST)
3,2011,Ford,F-150,Platinum,SuperCrew,automatic,1ftfw1e66bfc05737,mn,3.2,34884.0,black,gray,"ford motor credit company,llc",29600,28600,Tue Dec 23 2014 14:00:00 GMT-0800 (PST)
4,2013,Ford,Focus,SE,Sedan,automatic,1fadp3f25dl326975,tx,1.9,43191.0,blue,gray,santander consumer,10300,9800,Wed Feb 11 2015 02:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2006,Mercury,Milan,Base,Sedan,automatic,3mefm07z56r627058,fl,3,135307.0,silver,black,orlando hyundai,2175,1400,Tue Feb 10 2015 10:00:00 GMT-0800 (PST)
96,2014,Toyota,Corolla,LE,Sedan,automatic,5yfburhe5ep021117,ne,3.5,30779.0,gray,gray,avis corporation,12400,12600,Thu Jun 18 2015 04:30:00 GMT-0700 (PDT)
97,2007,Chevrolet,Tahoe,LT,SUV,automatic,1gnfk13067j177234,mo,3.7,247414.0,black,gray,primeritus remarketing/fast line auto sales inc,9875,10200,Tue Jan 13 2015 11:00:00 GMT-0800 (PST)
98,2014,Nissan,Frontier,SV,Crew Cab,NaN,1n6ad0er3en719511,nv,4.9,20159.0,white,gray,unique autos,20300,21000,Wed Dec 31 2014 12:00:00 GMT-0800 (PST)


In [27]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline

In [29]:
# Define the missing value imputation strategies
numeric_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Define the transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', numeric_imputer),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', categorical_imputer),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Fit the preprocessor on the training data
preprocessor.fit(X_test)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['year', 'odometer']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['make', 'model', 'trim', 'body',
                                  'transmission', 'vin', 'state', 'condition',
                                  'color', 'interior', 'seller', 'saledate'])])

In [30]:
# Create the SGDRegressor
regressor = SGDRegressor(loss='squared_error', random_state=100)

In [31]:
# Mini-batch training loop
batch_size = 50
num_batches = 100

for _ in range(num_batches):
    batch_df = next(gen)
    X_batch = batch_df.drop('sellingprice', axis=1)
    y_batch = batch_df['sellingprice']
    
    # Apply preprocessor to the mini-batch
    X_batch_processed = preprocessor.transform(X_batch)
    
    # Handle missing values in the mini-batch
    X_batch_processed = np.nan_to_num(X_batch_processed)
    
    regressor.partial_fit(X_batch_processed, y_batch)

In [32]:
# Save the model
import joblib
joblib.dump(regressor, 'regressor_model.joblib')

# Load the model
loaded_regressor = joblib.load('regressor_model.joblib')

# Preprocess the test data
X_test_processed = preprocessor.transform(X_test)

# Test the model on X_test
y_pred = loaded_regressor.predict(X_test_processed)

# Print the predicted values
print(y_pred)

[11187.17637994 20082.41424365 19715.94006659 ... 18361.30271693
 18426.78464946 18074.04560743]
